In [27]:
import numpy as np
import pandas as pd
import re

import json
import gzip
import ast

pd.options.mode.chained_assignment = None

In [28]:
with gzip.GzipFile('t.json.gz', 'r') as fin:
    json_bytes = fin.readlines()          

data = []
for lines in json_bytes:
    temp = lines.decode('utf-8')[:-1]
    temp = re.sub("null","None",temp)
    d = ast.literal_eval(temp)
    
    data.append(d)

In [29]:
t = pd.DataFrame(data,columns = ['category','subcategory','available_price','crawl_date','http_status','mrp','pack_size','title','urlh'])

In [30]:
with gzip.GzipFile('y.json.gz', 'r') as fin:
    json_bytes = fin.readlines()          

data = []
for lines in json_bytes:
    temp = lines.decode('utf-8')[:-1]
    temp = re.sub("null","None",temp)
    d = ast.literal_eval(temp)
    
    data.append(d)

In [31]:
y = pd.DataFrame(data,columns = ['category','subcategory','available_price','crawl_date','http_status','mrp','pack_size','title','urlh'])

<b><i>1. No of URLH which are overlapping

In [6]:
n_overlap = (t['urlh'].isin(y['urlh'])).value_counts()[True]

In [7]:
print("Number of urlh which are overlapping : " + str(n_overlap))

Number of urlh which are overlapping : 140414


<b><i>2. For all the URLH which are overlapping, calculate the price difference (wrt available_price) if there is any between yesterday's and today's crawls. There might be duplicate URLHs in which case you can choose the first valid (with http_status 200) record.

In [14]:
t = t.dropna(subset=['mrp'])
y = y.dropna(subset=['mrp'])
t['mrp'] = t['mrp'].apply(lambda x : float(x))
y['mrp'] = y['mrp'].apply(lambda x : float(x))

In [15]:
Mrp_x = t[['mrp','urlh']][t['http_status'] == '200']
Mrp_y = y[['mrp','urlh']][y['http_status'] == '200']

In [16]:
Mrp_x = Mrp_x.drop_duplicates('urlh',keep='first')
Mrp_y = Mrp_y.drop_duplicates('urlh',keep='first')

In [17]:
data = []
count = 0
import math
for index,row in Mrp_x.iterrows():
    price_diff ={}
    mrp_x = row['mrp']
    urlh = row['urlh']
    mrp_y = Mrp_y['mrp'][Mrp_y['urlh'] == urlh]
    
    if math.isnan(mrp_x) or mrp_x ==None:
        if len(mrp_y.values) == 0 or mrp_y.values == None:
            diff = 'NA'
        else:
            diff = mrp_y
            
    elif len(mrp_y.values) == 0  or mrp_y.values == None:
        if math.isnan(mrp_x) or mrp_x ==None:
            diff = 'NA'
        else:
            diff = mrp_x
        
    else:
        mrp_y = float(mrp_y.values[0])
        count +=1
        diff = mrp_x - mrp_y
        price_diff[urlh] = diff
    data.append(price_diff)
    if count % 2000 == 0:
        print(count)


2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000
82000
84000
86000
88000
90000
92000
94000
96000
98000
100000
102000
104000
106000
108000
110000
112000
114000
116000
118000
120000
122000
124000
126000
128000


In [23]:
Price_diff = data

In [25]:
Price_diff

[{'1ef0f41ebcee030a35440405f2f8b0165c719c89': 0.0},
 {'c566570abdbe95b5e6a78eb8b3b2424108bf36a2': 0.0},
 {'3b22b28b351dec75664b17fcf586023ddc21704c': 0.0},
 {'7ff787f49e137237054f95e7cb5b0f8611e7f6fb': 0.0},
 {'a7ad16e71257855c033aad46eb5082a2153b97c6': 0.0},
 {'7842869e31f564268680d898976fad8dafb135aa': 0.0},
 {'6397ec7c03a550bb4b4f508bc58dbd92adae6d51': 0.0},
 {'1bfe149f2392841ba7a71a6ef7c366cc982a49ca': 0.0},
 {'4dd425297b7e0ce4db883350d2133fb77a6fe533': 0.0},
 {'cc7f63df1d280c4feeb6574c0d4fb1f81732f5fe': 0.0},
 {'f46e0aab5e45ed77fadc6ceff04a4c891e97dd79': 0.0},
 {'5a98a19ab531c749442b5cd91b63199358047874': 0.0},
 {'1e0975914347b9d67c20bfa3376662ca33dacb8e': 0.0},
 {'b47f120a137c8874fddabba6424fb13c210ed177': 0.0},
 {'15f7f30a5fceb733c060c5064ad840cfcfee9f7c': 0.0},
 {'5405bef4520ce36da0eac8354c5662cdb0818d2d': 0.0},
 {'27aa381cbf21b28ba797dd1650f5cdfdebe4fe0b': 0.0},
 {'aef4e33f0eda81017d7e101d1f398db888d53748': 0.0},
 {'70f28cae91721337c525619496c3b1e5a1840f1d': 0.0},
 {'98d190967

<i><b>3.No of Unique categories in both files.

In [51]:
t_category = t['category'].value_counts()
y_category = y['category'].value_counts()

In [52]:
t_category

Personal Care                    30934
Snacks                           13326
Pantry                           12057
Beverages                        11600
Household                        10869
Frozen                            8355
Dairy & Eggs                      6295
Christmas                         5392
Canned Goods                      4520
Deli                              4103
Bakery                            3566
Dry Goods & Pasta                 3304
Meat & Seafood                    3092
Produce                           3034
Pets                              2974
Red Wine                          2678
Babies                            2572
Breakfast                         2440
International                     1773
Bulk                              1300
Ale Beer                          1070
White Wine                         940
Vodka                              596
Alcohol                            567
Liqueurs/Cordials/Schnapps         485
Champagne & Sparkling Win

In [53]:
y_category

Personal Care                    30648
Snacks                           13253
Pantry                           12066
Beverages                        11586
Household                        10727
Frozen                            8356
Dairy & Eggs                      6291
Christmas                         5396
Canned Goods                      4515
Deli                              4128
Bakery                            3570
Dry Goods & Pasta                 3299
Meat & Seafood                    3116
Produce                           3043
Pets                              2977
Red Wine                          2672
Babies                            2567
Breakfast                         2438
International                     1771
Bulk                              1300
Ale Beer                          1038
White Wine                         939
Vodka                              596
Alcohol                            567
Liqueurs/Cordials/Schnapps         487
Champagne & Sparkling Win

<i><b>4. List of categories which is not overlapping.

In [57]:
overlap_cat = t_category[t_category.isin(y_category)]

In [58]:
overlap_cat

Bulk                             1300
Vodka                             596
Alcohol                           567
Seasonal/Special Release Beer     218
Brandy & Cognac                   198
Glassware & Accessories           133
Holiday Favorites                  92
Fruit Wine                         63
Gift Baskets & Sets                 8
Name: category, dtype: int64

In [59]:
no_overlap_cat = t_category[~t_category.isin(y_category)]

In [60]:
no_overlap_cat

Personal Care                 30934
Snacks                        13326
Pantry                        12057
Beverages                     11600
Household                     10869
Frozen                         8355
Dairy & Eggs                   6295
Christmas                      5392
Canned Goods                   4520
Deli                           4103
Bakery                         3566
Dry Goods & Pasta              3304
Meat & Seafood                 3092
Produce                        3034
Pets                           2974
Red Wine                       2678
Babies                         2572
Breakfast                      2440
International                  1773
Ale Beer                       1070
White Wine                      940
Liqueurs/Cordials/Schnapps      485
Champagne & Sparkling Wine      460
American Whiskey                420
Lager Beer                      395
Rum                             284
Dessert & Fortified Wine        284
Tequila                     

<b><i>5. Generate the stats with count for all taxonomies (taxonomy is concatenation of category and subcategory separated by " > ").<br>
Eg:<br>
Cat1 > Subcat1: 3500<br>
Cat1 > Subcat2: 2000<br>
Cat2 > Subcat3: 8900

In [61]:
cat = t[['category','subcategory']]

In [62]:
taxonomy = cat['category'].groupby([cat['category'],cat['subcategory']]).count()

In [63]:
taxonomy

category                   subcategory                 
Alcohol                    Beers & Coolers                  197
                           Cocktail Mixes                     7
                           Red Wines                        224
                           Rosés                             26
                           Specialty Wines & Champagnes      45
                           Spirits                            5
                           White Wines                       63
Ale Beer                   Amber/Red Ale                     63
                           Belgian-Style Ale                142
                           Blond Ale                         59
                           Brown Ale                         24
                           IPA (India Pale Ale)             389
                           Pale Ale                         115
                           Porter                            38
                           Stout                

<b><i>6. Generate a new file where mrp is normalized. If there is a 0 or a non-float value or the key doesn't exist, make it "NA".

In [65]:
mrp_col = t['mrp']

In [66]:
mrp_list = [float(x) for x in mrp_col if x != None]
x_max = max(mrp_list)
x_min = min(mrp_list)

In [67]:
def normalize_mrp(x):
    if x != None:
        x = float(x)
    else:
        return 'NA'
    s = x_max -x_min
    x_new = (x - x_min) / s
    return x_new

In [68]:
Normalized_t = t

In [69]:
Normalized_t['mrp'] = Normalized_t['mrp'].apply(lambda x : normalize_mrp(x))

In [70]:
Normalized_t

,category,subcategory,available_price,crawl_date,http_status,mrp,pack_size,title,urlh
0,Household,Trash Bags & Liners,5.99,20180727,200,0,30 ct,Seventh Generation 13 Gallon Extra Strong Flap...,1ef0f41ebcee030a35440405f2f8b0165c719c89
1,Household,Trash Bags & Liners,5.49,20180727,200,0,25 ct,BioBag Food Scrap Bags Small 3 Gallon - 25 CT ...,c566570abdbe95b5e6a78eb8b3b2424108bf36a2
2,Household,Trash Bags & Liners,5.69,20180727,200,0,30.0 ct,If You Care Certified Compostable Food Waste B...,3b22b28b351dec75664b17fcf586023ddc21704c
3,Household,Trash Bags & Liners,5.99,20180727,200,0,12 ct,BioBag Food Scrap Bags Tall Kitchen 13 Gallon ...,7ff787f49e137237054f95e7cb5b0f8611e7f6fb
4,Household,Trash Bags & Liners,5.99,20180727,200,0,20 ct,Seventh Generation 13 Gal with Drawstring Clos...,a7ad16e71257855c033aad46eb5082a2153b97c6
5,Household,Paper Goods,3.99,20180727,200,0,3 ct,Field Day 100% Recycled Paper Towels - 3 ct,7842869e31f564268680d898976fad8dafb135aa
6,Household,Paper Goods,11.99,20180727,200,0,12 ct,Seventh Generation 2-Ply 100% Recycled Bathroo...,6397ec7c03a550bb4b4f508bc58dbd92adae6d51
7,Household,Paper Goods,7.99,20180727,200,0.089909,12 ct,Field Day 100% Recycled Bath Tissue - 12 ct,1bfe149f2392841ba7a71a6ef7c366cc982a49ca
8,Household,Paper Goods,2.69,20180727,200,0.029903,82.5 sq ft,Seventh Generation 100% Unbleached Recycled Pa...,4dd425297b7e0ce4db883350d2133fb77a6fe533
9,Household,Paper Goods,4.89,20180727,200,0.0529053,2 ct,Seventh Generation 2-Ply Right Size 100% Recyc...,cc7f63df1d280c4feeb6574c0d4fb1f81732f5fe
